### 요약(Summary) Evaluators
일부 메트릭은 실험의 개별 실행이 아닌 전체 실험 수준에서만 정의할 수 있습니다.

예를 들어, 데이터 세트에서 시작된 실험의 **모든 실행에 걸쳐 분류자의 평가 점수**를 계산 하고 싶을 수 있습니다.

이를 `summary_evaluators` 라고 합니다.

이러한 평가자는 하나의 Run과 Example 대신 각각의 목록을 가져옵니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [1]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


### RAG 성능 테스트를 위한 함수 정의
테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [3]:
from myrag import PDFRAG

def ask_question_with_llm(llm):
    rag=PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )
    retriever = rag.create_retriever()

    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs:dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"])
        }
    return _ask_question

OpenAIRelevanceGrader 는 질문(Question), 컨텍스트(Context), 답변(Answer) 가 관련성이 있는지 여부를 평가하는 데 사용됩니다.

- target="retrieval-question": 질문과 컨텍스트가 관련성이 있는지 여부를 평가합니다.
- target="retrieval-answer": 답변과 컨텍스트가 관련성이 있는지 여부를 평가합니다.


In [10]:
from langchain_teddynote.evaluator import OpenAIRelevanceGrader
from langchain_openai import ChatOpenAI

In [11]:
gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model="EEVE-Korean-10.8B:latest"))

NameError: name 'ChatOllama' is not defined

In [4]:



rq_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-question"
).create()

ra_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-answer"
).create()

### 관련성(Relevance) 평가를 종합하는 Summary Evaluator

In [5]:
from typing import List
from langsmith.schemas import Example, Run
from langsmith.evaluation import evaluate


def relevance_score_summary_evaluator(runs: List[Run], examples: List[Example]) -> dict:
    rq_scores = 0  # 질문 관련성 점수
    ra_scores = 0  # 답변 관련성 점수

    for run, example in zip(runs, examples):
        question = example.inputs["question"]
        context = run.outputs["context"]
        prediction = run.outputs["answer"]

        # 질문 관련성 평가
        rq_score = rq_grader.invoke(
            {
                "input": question,
                "context": context,
            }
        )
        # 답변 관련성 평가
        ra_score = ra_grader.invoke(
            {
                "input": prediction,
                "context": context,
            }
        )

        # 관련성 점수 누적
        if rq_score.score == "yes":
            rq_scores += 1
        if ra_score.score == "yes":
            ra_scores += 1

    # 최종 관련성 점수 계산 (질문 관련성과 답변 관련성의 평균)
    final_score = ((rq_scores / len(runs)) + (ra_scores / len(runs))) / 2

    return {"key": "relevance_score", "score": final_score}

In [9]:
# 평가 실행
dataset_name = "RAG_EVAL_DATASET_teddynote"

experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    ollama_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

View the evaluation results for experiment: 'SUMMARY_EVAL-8fa0b230' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=195c32b6-c7a5-4c23-9f87-e90d83088f57




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running summary evaluator <function relevance_score_summary_evaluator at 0x0000027D87F68400>: 'question'


NameError: name 'ollama_chain' is not defined